### Imports

In [19]:
from pathlib import Path
import gym
import gym.spaces 
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from collections import deque, namedtuple
from tqdm import tqdm
import datetime
import random
import pickle
  
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

device = 'cuda' if torch.cuda.is_available() else 'cpu'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
def set_seed(seed, env):
    env.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

### Wrappers

In [3]:
class NoopResetEnv(gym.Wrapper):
    """
    Realiza un número random de "NOOP" al invocar reset().
    """
    def __init__(self, env, noop_max=30):
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.noop_action = 0
        assert self.env.unwrapped.get_action_meanings()[0] == 'NOOP'
        
    def reset(self):
        self.env.reset()
        noops = self.env.unwrapped.np_random.integers(1, self.noop_max+1)
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset()
        return obs

In [4]:
class MaxAndSkipEnv(gym.Wrapper):
    """
    Salta un número de frames y regresa el valor promedio de cada pixel.
    """
    def __init__(self, env, skip=4):
        gym.Wrapper.__init__(self, env)
        self._skip = skip
        self._obs_buffer = np.zeros((self._skip,) + self.env.observation_space.shape, dtype=np.uint8)
        
    def step(self, action):
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, _, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs            
            total_reward += reward
            if done:
                break
        max_frame = self._obs_buffer.max(axis=0)
        return max_frame, total_reward, done, info
    
    def reset(self):
        obs = self.env.reset()
        return obs

In [5]:
class TimeLimit(gym.Wrapper):
    """
    Termina el episodio después de un número de pasos.
    Evita que los ambientes entren en un loop o sin moverse.
    """
    def __init__(self, env, max_episode_steps=None):
        super(TimeLimit, self).__init__(env)
        self._max_episode_steps = max_episode_steps
        self._elapsed_steps = 0
        
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self._elapsed_steps += 1
        if self._elapsed_steps >= self._max_episode_steps:
            done = True
            info['TimeLimit.truncated'] = True
        return obs, reward, done, info

    def reset(self):
        self._elapsed_steps = 0
        obs = self.env.reset()
        return obs
        

In [6]:
class FireResetEnv(gym.Wrapper):
    """
    Realiza la acción "FIRE" para iniciar los juegos que lo requieran.
    """
    def __init__(self, env):
        gym.Wrapper.__init__(self, env)
        self.lives = self.env.ale.lives()
        assert self.env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(self.env.unwrapped.get_action_meanings()) >= 3
    
    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs
    
    def step(self, action):
        if self.lives > self.env.ale.lives():
            self.lives = self.env.ale.lives()
            action = 1
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, info

In [7]:
class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_r=-1, max_r=1):
        super().__init__(env)
        self.min_r = min_r
        self.max_r = max_r
        
    def reward(self, reward):
        if reward < 0:
            return -1
        elif reward > 0:
            return 1
        else:
            return 0
        
    def step(self, action):
        obs, rew, done, info = self.env.step(action)
        return obs, self.reward(rew), done, info

In [8]:
class WarpFrame(gym.ObservationWrapper):
    """
    Reescala las imágenes a 84x84 y las pasa de RGB a gris.
    """
    def __init__(self, env, width=84, height=84):
        super().__init__(env)
        self._width = width
        self._height = height
        num_colors = 1
        
        new_space = gym.spaces.Box(
            low = 0,
            high = 255,
            shape = (self._height, self._width, num_colors),
            dtype = np.uint8,
        )
        original_space = self.observation_space
        self.observation_space = new_space
        assert original_space.dtype == np.uint8 and len(original_space.shape) == 3
        
    def observation(self, obs):
        frame = obs
        frame = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
        frame = cv.resize(
            frame, (self._width, self._height), interpolation=cv.INTER_AREA
        )
        obs = frame
        return obs
    
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        return self.observation(obs), reward, done, info
    
    def reset(self):
        obs = self.env.reset()
        return self.observation(obs)

In [9]:
class ScaledFloatFrame(gym.ObservationWrapper):
    """
    Reescala de 0-255 a 0-1.
    """
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(
            low = 0,
            high = 255,
            shape = self.env.observation_space.shape,
            dtype = np.float32,
        )
    
    def observation(self, observation):
        return np.array(observation).astype(np.float32) / 255.0
        
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        return self.observation(obs), reward, done, info
    
    def reset(self):
        obs = self.env.reset()
        return self.observation(obs)

In [10]:
class FrameStack(gym.Wrapper):
    """
    Apila los últimos k frames.
    """
    def __init__(self, env, k=4):
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        self.observation_space = gym.spaces.Box(
            low = 0,
            high = 255,
            shape = (4,84,84),
            dtype = self.env.observation_space.dtype,
        )
    
    def reset(self):
        obs = self.env.reset()
        for _ in range(self.k):
            self.frames.append(obs)
        return self._get_obs()
    
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_obs(), reward, done, info
    
    def _get_obs(self):
        return self.frames

### Creación del ambiente

In [11]:
def make_atari(env_id, frames=4, max_episode_steps=None, noop_max=30, skip=4):
    """
    Crea el ambiente especificado, pasándolo por los Wrappers especificados.
    """
    env = gym.make(env_id, render_mode=None)
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max)
    env = MaxAndSkipEnv(env, skip)
    if max_episode_steps is not None:
        env = TimeLimit(env, max_episode_steps=max_episode_steps)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = ClipReward(env)
    env = WarpFrame(env)
    env = ScaledFloatFrame(env)
    env = FrameStack(env, frames)
    return env

##### TEST

GAME = 'MsPacmanNoFrameskip-v4'
env = make_atari(GAME)

start = env.reset()

start[0].shape

plt.imshow(start[0])
plt.show()

state, _, _, _ = env.step(0)

plt.imshow(state[0])
plt.show()

### DQN

In [12]:
class DQN(nn.Module):
    """
    Red Profunda de Aprendizaje Q (Deep Q Network).
    """
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
        
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1,*shape))
        return int(np.prod(o.size()))
    
    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)      

net = DQN(env.observation_space.shape, env.action_space.n).to(device)
net

### Experience Replay

In [13]:
Experience = namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'next_state'])

class ExperienceReplay:
    """
    Almacena experiencias pasadas que han sido observadas por el agente.
    Las muestras obtenidas sirven para entrenar la red, buscando minimizar el efecto que tiene la correlación entre pasos.
    """
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def __len__(self):
        return len(self.buffer)
    
    def append(self, *args):
        self.buffer.append(Experience(*args))
        
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

### Agent

In [14]:
class Agent:
    """
     
    """
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()
        
    def _reset(self):
        self.state = self.env.reset()
        self.total_reward = 0.0
        
    def play_step(self, net, epsilon=0.0, device='cuda'):
        done_reward = None
        
        if np.random.random() < epsilon:
            action = self.env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1) # Devuelve el índice de la acción
            action = int(act_v.item())
        
        new_state, reward, done, _ = self.env.step(action)
        self.total_reward += reward

        self.exp_buffer.append(self.state, action, reward, done, new_state)
        self.state = new_state
        
        if done:
            done_reward = self.total_reward
            self._reset()
        
        return done_reward

### Función de entrenamiento

In [43]:
def training(env_name, replay_memory_size=1_000_000, max_frames=50_000_000, gamma=0.99, batch_size=32,  \
             learning_rate=0.00025, momentum=0.95, min_gradient=0.1, sync_target_frames=10_000, \
             replay_start_size=50_000, eps_start=1, eps_min=0.1, seed=2109, device='cuda', verbose=True):
    """
    Función de entrenamiento.
    """
    path = "dicts/" + env_name 
    Path(path).mkdir(parents=True, exist_ok=True)
    
    env = make_atari(env_name)
    buffer = ExperienceReplay(replay_memory_size)
    agent = Agent(env, buffer)
    set_seed(seed=seed, env=env)
    
    net        = DQN(env.observation_space.shape, env.action_space.n).to(device)
    target_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
    writer = SummaryWriter(comment='-' + env_name)
    
    epsilon = eps_start
    eps_decay = (eps_start - eps_min) / replay_memory_size
    
    optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, momentum=momentum, eps=min_gradient)
    total_rewards = []
    
    best_mean_reward = None
    start_time = datetime.datetime.now()
    
    for frame in tqdm(range(1, max_frames+1)):
        epsilon = max(epsilon-eps_decay, eps_min)
        
        reward = agent.play_step(net, epsilon, device=device)
        if reward is not None:
            total_rewards.append(reward)
            mean_reward = np.mean(total_rewards[-100:])
            
            time_passed = datetime.datetime.now() - start_time
            
            #writer.add_scalar("epsilon", epsilon, frame)
            writer.add_scalar("reward_100", mean_reward, frame)
            writer.add_scalar("reward", reward, frame)
            
            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), path + "/" + env_name + "_best.dat")
                best_mean_reward = mean_reward
                
        if len(buffer) < replay_start_size:
            continue
            
        sardn = buffer.sample(batch_size)
        batch = Experience(*zip(*sardn))
        
        states_v = torch.tensor(np.array(batch.state)).to(device)
        next_states_v = torch.tensor(np.array(batch.next_state)).to(device)
        actions_v = torch.tensor(batch.action).to(device)
        rewards_v = torch.tensor(batch.reward).to(device)
        done_mask = torch.BoolTensor(batch.done).to(device)
        
        state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
        next_state_values = target_net(next_states_v).max(1)[0]
        next_state_values[done_mask] = 0.0
        next_state_values = next_state_values.detach()
        expected_state_action_values = next_state_values*gamma + rewards_v
        
        loss_t = nn.MSELoss()(state_action_values, expected_state_action_values) # MSELoss()(input,target)
        
        optimizer.zero_grad()
        loss_t.backward()
        optimizer.step()
        
        if (frame + 1) % sync_target_frames == 0:
            target_net.load_state_dict(net.state_dict())

        if (frame + 1) % (max_frames / 50) == 0:
            if verbose:
                print("{}:  {} games, best result {:.3f}, mean reward {:.3f}, eps {:.2f}, time {}".format(
                    frame + 1, len(total_rewards), max(total_rewards), mean_reward, epsilon, time_passed))
        if (frame+1) % (max_frames/10) == 0:
            torch.save(net.state_dict(), path + "/" + env_name + "_" + str(int((frame+1)/(max_frames/10))) + ".dat")

    print("Training finished")
    print("{}:  {} games, mean reward {:.3f}, eps {:.2f}, time {}".format(
            frame + 1, len(total_rewards), mean_reward, epsilon, time_passed))
         
    writer.close()
    pkl_file = "dicts/" + env_name + "/" + env_name + ".pkl"
    with open(pkl_file, 'wb+') as f:
        pickle.dump(total_rewards, f)
    return total_rewards

### TRAINING

In [44]:
GAME_NAME = "MsPacmanNoFrameskip-v4"

In [45]:
pacman = training(env_name=GAME_NAME)

100%|██████████| 5000/5000 [00:04<00:00, 1103.60it/s]

Training finished
5001:  10 games, mean reward 20.200, eps 1.00, time 0:00:04.280481


In [86]:
envs = np.array(list(gym.envs.registry.keys()))
envs_mask1 = ['ramNoFrameskip' not in k for k in envs]
envs = envs[envs_mask1]
envs_mask2 = ['NoFrameskip-v4' in k for k in envs]
valid_envs = envs[envs_mask2]

In [87]:
valid_envs

array(['AdventureNoFrameskip-v4', 'AirRaidNoFrameskip-v4',
       'AlienNoFrameskip-v4', 'AmidarNoFrameskip-v4',
       'AssaultNoFrameskip-v4', 'AsterixNoFrameskip-v4',
       'AsteroidsNoFrameskip-v4', 'AtlantisNoFrameskip-v4',
       'BankHeistNoFrameskip-v4', 'BattleZoneNoFrameskip-v4',
       'BeamRiderNoFrameskip-v4', 'BerzerkNoFrameskip-v4',
       'BowlingNoFrameskip-v4', 'BoxingNoFrameskip-v4',
       'BreakoutNoFrameskip-v4', 'CarnivalNoFrameskip-v4',
       'CentipedeNoFrameskip-v4', 'ChopperCommandNoFrameskip-v4',
       'CrazyClimberNoFrameskip-v4', 'DefenderNoFrameskip-v4',
       'DemonAttackNoFrameskip-v4', 'DoubleDunkNoFrameskip-v4',
       'ElevatorActionNoFrameskip-v4', 'EnduroNoFrameskip-v4',
       'FishingDerbyNoFrameskip-v4', 'FreewayNoFrameskip-v4',
       'FrostbiteNoFrameskip-v4', 'GopherNoFrameskip-v4',
       'GravitarNoFrameskip-v4', 'HeroNoFrameskip-v4',
       'IceHockeyNoFrameskip-v4', 'JamesbondNoFrameskip-v4',
       'JourneyEscapeNoFrameskip-v4', 'Kan

#### Muestreo

In [ ]:
def sample(env_name, n_samples=20):
    '''
    Obtiene 'n_samples' muestras de la red entrenada.
    '''
    model = 'dicts/' + env_name + "/" + env_name + '_best.dat'

    env = make_atari(env_name)
    net = DQN(env.observation_space.shape, env.action_space.n)
    net.load_state_dict(torch.load(model, map_location=lambda storage, loc: storage))
    
    rewards = np.zeros(n_samples)

    for i in range(n_samples):
    
        state = env.reset()
        total_reward = 0.0
        
        while True:
            
            state_v = torch.tensor(np.array([state], copy=False))
            q_vals = net(state_v).data.numpy()[0]
            action = np.argmax(q_vals)

            state, reward, done, _ = env.step(action)
            total_reward += reward

            if done:
                break

        rewards[i] = total_reward
        
    return rewards

In [ ]:
sample(GAME_NAME, n_samples=20)

array([-21., -21., -21., -21., -21., -21., -21., -21., -21., -21., -21.,
       -21., -21., -21., -21., -21., -21., -21., -21., -21.])